In [19]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [20]:
with open('intents.json') as file:
    data = json.load(file)

In [21]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [22]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [23]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 42)                714       
                                                                 
Total params: 17,258
Trainable params: 17,258
Non-trainable params: 0
__________________________________________________

In [26]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
12/12 [==============================] - 1s 5ms/step - loss: 3.7353 - accuracy: 0.0375
Epoch 2/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7295 - accuracy: 0.0322
Epoch 3/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7234 - accuracy: 0.0456
Epoch 4/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7164 - accuracy: 0.0536
Epoch 5/550
12/12 [==============================] - 0s 6ms/step - loss: 3.7075 - accuracy: 0.0509
Epoch 6/550
12/12 [==============================] - 0s 6ms/step - loss: 3.6972 - accuracy: 0.0429
Epoch 7/550
12/12 [==============================] - 0s 7ms/step - loss: 3.6837 - accuracy: 0.0429
Epoch 8/550
12/12 [==============================] - 0s 7ms/step - loss: 3.6693 - accuracy: 0.0456
Epoch 9/550
12/12 [==============================] - 0s 6ms/step - loss: 3.6521 - accuracy: 0.0563
Epoch 10/550
12/12 [==============================] - 0s 7ms/step - loss: 3.6360 - accuracy: 0.0456
Epoch 11/

In [27]:
# saving model
model.save("chat_model_new.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)